In [3]:
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  2246k      0 --:--:-- --:--:-- --:--:-- 2255k


In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
len(text)

1115394

In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size:', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [8]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[ch] for ch in x])

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
data.size(), data[:10]

(torch.Size([1115394]), tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]))

In [10]:
n = int(len(data)*0.9)
train, valid = data[:n], data[n:]
train.size(), valid.size()

(torch.Size([1003854]), torch.Size([111540]))

In [11]:
block_size = 8
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train if split == 'train' else valid
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
xb.size(), yb.size()

(torch.Size([4, 8]), torch.Size([4, 8]))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, x, targets = None):   
        logits = self.token_embedding_table(x)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.size()
            logits = logits.view(B*T,C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits,loss = m(xb, yb)
logits.size(), loss

print(decode(m.generate(torch.zeros(1,1, dtype=torch.long), 100)[0].tolist()))



Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [33]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [35]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
loss

tensor(2.4286, grad_fn=<NllLossBackward0>)

In [36]:
print(decode(m.generate(torch.zeros(1,1, dtype=torch.long), 500)[0].tolist()))


Te.
Harmatant y.
NGLI I wentoo n;
Burornn aler tho lyoueme onas; ncour'l ice,
WWhe buring, yme perorshy alg, wa,

Foury tovesso ard.
DUCore he,
DWhe Ye pl, INCali'souoormb, zeali't tof he st me, wond he s allers lldele ma wa t hiello tromedo,
HIn I I iny y and, llang?
Cllcasutoum;


OVIs y' St.
havequtish stothar's.
Whand ap wip, t grd indatone;
FRDWellor'
ARCEXELLAs peer s bemif k sh:
Ayoourtot mavFo I touss mer: ty, fo atck I w tou II:
ANoury y h?
Whal'D mowimives rtigalomys t, hs lende! ayway


In [ ]:
xbow = 